In [3]:
# Load C traces
import pandas as pd
import json
import re
from pprint import pp
from tree_sitter_languages import get_parser
import os
from lxml import etree
from pathlib import Path
import tqdm
from Sampling_Block import *

#files = list(Path('C/Traces').glob("**/log_*.xml"))

# Define the base directory
base_dir = Path('/home/XXX/trace-modeling-oss-fuzz-c/trace_test/traces')

# List all directories ending with '_own'
own_dirs = [d for d in base_dir.glob("*_own") if d.is_dir()]

log_files = []

for trace_dir in own_dirs:
    log_files.extend(Path(trace_dir).glob("**/log_*.xml"))

print(len(log_files))

39034


In [4]:
import json
from filelock import FileLock 

class GlobalFunctionTracker:
    """Persistent function tracker using file storage"""
    def __init__(self, cache_file=".function_cache.json"):
        self.cache_file = cache_file
        self.lock = FileLock(f"{cache_file}.lock")
        self.cache = set()
        self._load_cache()
    
    def _load_cache(self):
        try:
            with self.lock:
                if os.path.exists(self.cache_file):
                    with open(self.cache_file, 'r') as f:
                        self.cache = set(json.load(f))
        except Exception as e:
            print(f"Warning: Could not load cache - {e}")
            self.cache = set()
    
    def _save_cache(self):
        try:
            with self.lock:
                with open(self.cache_file, 'w') as f:
                    json.dump(list(self.cache), f)
        except Exception as e:
            print(f"Warning: Could not save cache - {e}")
    
    def add(self, func_signature):
        """Add a function signature to the global tracker"""
        if func_signature not in self.cache:
            self.cache.add(func_signature)
            self._save_cache()
            return True
        return False
    
    def __contains__(self, func_signature):
        return func_signature in self.cache

# Initialize the global tracker at module level
global_tracker = GlobalFunctionTracker()

def get_function_signature(fpath, src_file):
    """Generate a more robust function signature"""
    try:
        # Use both the log file and source file to generate signature
        with open(fpath, 'rb') as f:
            log_content = f.read(1024).decode('utf-8', errors='ignore')
        
        with open(src_file, 'rb') as f:
            src_content = f.read(2048).decode('utf-8', errors='ignore')
        
        # Create a signature from file metadata and content
        return f"{os.path.basename(fpath)}:{hash(log_content)}:{hash(src_content)}"
    except Exception as e:
        print(f"Error generating signature: {e}")
        return str(fpath)  # Fallback to filename

def process_single_file(fpath, output_dir="output_1"):
    src_file_path = Path(str(fpath).replace("log_", "src_"))
    
    try:
        # Get robust function signature
        func_signature = get_function_signature(fpath, src_file_path)
        
        # Check global tracker first
        if func_signature in global_tracker:
            return None
        
        # Process the file
        total_samples = process_xml_to_code(
            str(fpath),
            str(src_file_path),
            seen_signatures_global=global_tracker,  # Pass to inner function
            output_dir=output_dir
        )
        
        # Add to global tracker if processing succeeded
        if total_samples is not None:
            global_tracker.add(func_signature)
        
        return total_samples
    
    except Exception as e:
        print(f"Error processing {fpath}: {e}")
        return None


def process_files_in_batches(log_files, batch_size=8, output_dir="output1"):
    # Process in batches
    results = []
    with tqdm.tqdm(total=len(log_files)) as pbar:
        for i in range(0, len(log_files), batch_size):
            batch = log_files[i:i+batch_size]
            
            # Process batch (no need for shared set anymore)
            batch_results = []
            for fpath in batch:
                result = process_single_file(fpath, output_dir=output_dir)
                if result is not None:
                    batch_results.append(result)
                pbar.update(1)
            
            results.extend(batch_results)
            
            # Periodic status
            if i % (10*batch_size) == 0:
                print(f"\nProcessed {i}/{len(log_files)} files")
                print(f"Current unique count: {len(global_tracker.cache)}")
    
    print(f"\nCompleted. Total unique functions: {len(global_tracker.cache)}")
    return results

process_files_in_batches(log_files, batch_size=8)

  0%|          | 0/39034 [00:00<?, ?it/s]

Error processing source file: Source file not found: /home/XXX/trace-modeling-oss-fuzz-c/trace_test/traces/tarantool_own/trace_luaL_loadbuffer_fuzzer_exe/src_0.xml
Falling back to HTML parser for /home/XXX/trace-modeling-oss-fuzz-c/trace_test/traces/tarantool_own/trace_swim_proto_member_fuzzer_exe/log_0.xml


Error generating signature: [Errno 2] No such file or directory: '/home/XXX/trace-modeling-oss-fuzz-c/trace_test/traces/tarantool_own/trace_luaL_loadbuffer_fuzzer_exe/src_0.xml'


Error processing call: 'NoneType' object has no attribute 'splitlines'
  0%|          | 2/39034 [00:01<6:46:58,  1.60it/s]Falling back to HTML parser for /home/XXX/trace-modeling-oss-fuzz-c/trace_test/traces/tarantool_own/trace_csv_fuzzer_exe_exe/log_2.xml
Fatal error processing trace file /home/XXX/trace-modeling-oss-fuzz-c/trace_test/traces/tarantool_own/trace_csv_fuzzer_exe_exe/log_2.xml: 'NoneType' object has no attribute 'xpath'
Falling back to HTML parser for /home/XXX/trace-modeling-oss-fuzz-c/trace_test/traces/tarantool_own/trace_swim_proto_meta_fuzzer_exe/log_0.xml
  0%|          | 11/39034 [00:02<1:46:03,  6.13it/s]


Processed 0/39034 files
Current unique count: 24


...

100%|█████████▉| 38968/39034 [1:12:12<00:08,  7.35it/s]


Processed 38960/39034 files
Current unique count: 709


100%|██████████| 39034/39034 [1:12:24<00:00,  8.98it/s]


Completed. Total unique functions: 710


[]

In [5]:
import json

# Initialize counters
block_size_counts = {1: 0, 2: 0, 3: 0}

# Path to your JSON Lines file
file_path = "/home/XXX/Tracing/Annotation/C Benchmark Collection/RQ2/output1/block_analysis_c.jsonl"

try:
    with open(file_path, 'r') as file:
        for line in file:
            try:
                # Parse JSON from each line
                entry = json.loads(line)
                
                # Get block size (default to 0 if not found)
                block_size = entry.get("Block_Size", 0)
                
                # Update counters for sizes 1-3
                if block_size in block_size_counts:
                    block_size_counts[block_size] += 1
                    
            except json.JSONDecodeError:
                print(f"Skipping invalid JSON line: {line}")
                
except FileNotFoundError:
    print(f"Error: File not found at {file_path}")
except Exception as e:
    print(f"An error occurred: {str(e)}")

# Print results
print("Block Size Counts:")
for size, count in block_size_counts.items():
    print(f"Size {size}: {count} samples")

Block Size Counts:
Size 1: 26 samples
Size 2: 25 samples
Size 3: 30 samples
